In [1]:

# 📦 Required Libraries
import pandas as pd
import numpy as np
import finnhub
import time
from datetime import datetime, timedelta
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as XLImage
import os
import shutil

# 📡 Finnhub Setup
FINNHUB_API_KEY = "API key"  # <-- Replace with your real key
finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

# 📥 Load Ticker List
tickers_df = pd.read_excel("PowerX_Ticker_Only_Input_Template.xlsx")
tickers = tickers_df['Stock Symbol'].dropna().tolist()

# 📅 Fetch Data
today_unix = int(time.time())
six_months_ago = int((datetime.now() - timedelta(days=180)).timestamp())

def fetch_candles(symbol):
    try:
        res = finnhub_client.stock_candles(symbol, 'D', six_months_ago, today_unix)
        if res and res.get("s") == "ok":
            df = pd.DataFrame({
                'Date': pd.to_datetime(res['t'], unit='s'),
                'Open': res['o'],
                'High': res['h'],
                'Low': res['l'],
                'Close': res['c'],
                'Volume': res['v'],
                'Stock Symbol': symbol
            })
            return df
    except Exception as e:
        print(f"Error fetching {symbol}: {e}")
    return pd.DataFrame()

all_data = []
for ticker in tickers:
    stock_df = fetch_candles(ticker)
    if not stock_df.empty:
        all_data.append(stock_df)

combined_df = pd.concat(all_data, ignore_index=True)

# 📈 Calculate Indicators
def calculate_macd(df):
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD_Line'] = ema12 - ema26
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=9, adjust=False).mean()
    df['MACD_Histogram'] = df['MACD_Line'] - df['MACD_Signal']
    return df

def calculate_rsi(df, window=7):
    delta = df['Close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    df['RSI_7'] = 100 - (100 / (1 + rs))
    return df

def calculate_stochastic(df, k_window=14, d_window=3):
    low_min = df['Low'].rolling(window=k_window).min()
    high_max = df['High'].rolling(window=k_window).max()
    df['%K'] = (df['Close'] - low_min) / (high_max - low_min) * 100
    df['%D'] = df['%K'].rolling(window=d_window).mean()
    df['Stoch_Smoothed'] = df['%D'].rolling(window=d_window).mean()
    return df

def calculate_adr(df, window=7):
    df['ADR'] = (df['High'] - df['Low']).rolling(window=window).mean()
    return df

enhanced_data = []
for symbol in combined_df['Stock Symbol'].unique():
    stock_df = combined_df[combined_df['Stock Symbol'] == symbol].copy()
    stock_df = calculate_macd(stock_df)
    stock_df = calculate_rsi(stock_df)
    stock_df = calculate_stochastic(stock_df)
    stock_df = calculate_adr(stock_df)
    enhanced_data.append(stock_df)

enhanced_df = pd.concat(enhanced_data, ignore_index=True)

# 🚦 Detect Signals
def detect_powerx_signal(row):
    if (row['MACD_Histogram'] > 0) and (row['RSI_7'] > 50) and (row['Stoch_Smoothed'] > 50):
        return "BUY"
    elif (row['MACD_Histogram'] < 0) and (row['RSI_7'] < 50) and (row['Stoch_Smoothed'] < 50):
        return "SELL"
    else:
        return "NEUTRAL"

latest_signals = enhanced_df.sort_values('Date').groupby('Stock Symbol').tail(1).copy()
latest_signals['PowerX Signal'] = latest_signals.apply(detect_powerx_signal, axis=1)

# 🤖 Predict Future Closes
LOOKBACK_WINDOW = 30
FUTURE_DAYS = 5

def predict_future_close(stock_df):
    close_prices = stock_df['Close'].values.reshape(-1, 1)
    scaler = MinMaxScaler()
    scaled_close = scaler.fit_transform(close_prices)
    X_train, y_train = [], []
    for i in range(LOOKBACK_WINDOW, len(scaled_close) - FUTURE_DAYS):
        X_train.append(scaled_close[i-LOOKBACK_WINDOW:i])
        y_train.append(scaled_close[i:i+FUTURE_DAYS])
    if len(X_train) == 0:
        return []
    X_train, y_train = np.array(X_train), np.array(y_train)
    model = Sequential([LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)), Dense(FUTURE_DAYS)])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
    last_window = scaled_close[-LOOKBACK_WINDOW:]
    last_window = last_window.reshape(1, LOOKBACK_WINDOW, 1)
    predicted_scaled = model.predict(last_window)
    return scaler.inverse_transform(predicted_scaled.reshape(-1, 1)).flatten()

future_predictions = []
for symbol in enhanced_df['Stock Symbol'].unique():
    stock_data = enhanced_df[enhanced_df['Stock Symbol'] == symbol].copy()
    predicted_closes = predict_future_close(stock_data)
    if len(predicted_closes) == FUTURE_DAYS:
        future_dates = pd.date_range(start=latest_signals[latest_signals['Stock Symbol'] == symbol]['Date'].values[0] + np.timedelta64(1, 'D'), periods=FUTURE_DAYS)
        for date, close in zip(future_dates, predicted_closes):
            future_predictions.append({"Stock Symbol": symbol, "Date": date, "Predicted Close": close})

future_predictions_df = pd.DataFrame(future_predictions)

# 🔮 Assign Future Signals
future_signal_predictions = []
for symbol in future_predictions_df['Stock Symbol'].unique():
    latest_real_close = latest_signals[latest_signals['Stock Symbol'] == symbol]['Close'].values[0]
    symbol_future = future_predictions_df[future_predictions_df['Stock Symbol'] == symbol]
    for _, row in symbol_future.iterrows():
        future_close = row['Predicted Close']
        signal = "BUY" if future_close > latest_real_close * 1.005 else "SELL" if future_close < latest_real_close * 0.995 else "NEUTRAL"
        future_signal_predictions.append({"Stock Symbol": symbol, "Date": row['Date'], "Close": future_close, "Signal Type": signal})

future_signals_df = pd.DataFrame(future_signal_predictions)

# 📦 Merge Dashboard
today_dashboard = latest_signals.copy()
today_dashboard = today_dashboard.rename(columns={'Close': 'Close', 'PowerX Signal': 'Signal Type'})
today_dashboard['Reason'] = today_dashboard['Signal Type'].apply(lambda x: "Trend confirmed." if x == "BUY" else "Trend reversal." if x == "SELL" else "No clear trend.")
today_dashboard = today_dashboard[["Stock Symbol", "Date", "Close", "Signal Type", "Reason"]]

future_dashboard = future_signals_df.copy()
future_dashboard['Reason'] = future_dashboard['Signal Type'].apply(lambda x: "Projected upward." if x == "BUY" else "Projected downward." if x == "SELL" else "Projected sideway.")
future_dashboard = future_dashboard[["Stock Symbol", "Date", "Close", "Signal Type", "Reason"]]

combined_dashboard = pd.concat([today_dashboard, future_dashboard], ignore_index=True)
combined_dashboard = combined_dashboard.sort_values(by=["Stock Symbol", "Date"])

# 📑 Save Dashboard
dashboard_filename = "PowerX_Daily_Dashboard_Natural_Phrasing_Combined.xlsx"
combined_dashboard.to_excel(dashboard_filename, index=False)

# 📈 Insert Graphs into New Worksheet
wb = load_workbook(dashboard_filename)

if "PowerX_Charts" in wb.sheetnames:
    charts_ws = wb["PowerX_Charts"]
else:
    charts_ws = wb.create_sheet(title="PowerX_Charts")

graph_temp_folder = "temp_graphs_powerx"
os.makedirs(graph_temp_folder, exist_ok=True)

chart_row_position = 1
for symbol in enhanced_df['Stock Symbol'].unique():
    stock_hist = enhanced_df[enhanced_df['Stock Symbol'] == symbol]
    stock_future = future_predictions_df[future_predictions_df['Stock Symbol'] == symbol]
    if stock_hist.empty or stock_future.empty:
        continue
    plt.figure(figsize=(8, 4))
    plt.plot(stock_hist['Date'], stock_hist['Close'], label="Historical", color='blue')
    plt.plot(stock_future['Date'], stock_future['Predicted Close'], label="Predicted", color='green', linestyle='dashed')
    plt.title(f"{symbol} Close and Prediction")
    plt.xlabel("Date")
    plt.ylabel("Close Price")
    plt.legend()
    plt.grid(True)
    img_path = os.path.join(graph_temp_folder, f"{symbol}_chart.png")
    plt.savefig(img_path)
    plt.close()
    img = XLImage(img_path)
    img.anchor = f"A{chart_row_position}"
    charts_ws.add_image(img)
    chart_row_position += 22

# Save final
final_output = "PowerX_Daily_Dashboard_Natural_Phrasing_Combined_WithSeparateGraphs.xlsx"
wb.save(final_output)

# Clean temporary folder
if os.path.exists(graph_temp_folder):
    shutil.rmtree(graph_temp_folder)

print(f"✅ Final PowerX Screener with Graphs in new worksheet saved: {final_output}")


1/1 [==============================] - 0s 378ms/step
✅ Final PowerX Screener with Graphs in new worksheet saved: PowerX_Daily_Dashboard_Natural_Phrasing_Combined_WithSeparateGraphs.xlsx
